# Encoder-Decoder Translation with Keras

## The Model

In [2]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = './data/spa.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[1000:20000]

In [3]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [4]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [5]:
# Clean sentences
english_sentences = [clean_sentence(pair[0]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenize(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length spanish sentence: {}'.format(len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
spanish_vocab = len(spa_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(spanish_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length spanish sentence: 9
Maximum length english sentence: 5
Spanish vocabulary is of 7265 unique words
English vocabulary is of 3720 unique words


In [6]:
max_spanish_len = int(len(max(spa_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

spa_pad_sentence = pad_sequences(spa_text_tokenized, max_spanish_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [7]:
input_sequence = Input(shape=(max_spanish_len,))
embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)

In [8]:
encoder = LSTM(64, return_sequences=False)(embedding)

In [9]:
r_vec = RepeatVector(max_english_len)(encoder)

In [10]:
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)

In [11]:
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [12]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 128)            929920    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 5, 64)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 5, 64)             33024     
                                                                 
 time_distributed (TimeDist  (None, 5, 3720)           241800    
 ributed)                                                    

In [13]:
model_results = enc_dec_model.fit(spa_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)

Epoch 1/100
634/634 [==============================] - 30s 38ms/step - loss: 4.5747 - accuracy: 0.3593
Epoch 2/100
634/634 [==============================] - 25s 39ms/step - loss: 3.9379 - accuracy: 0.3942
Epoch 3/100
634/634 [==============================] - 24s 38ms/step - loss: 3.7561 - accuracy: 0.4029
Epoch 4/100
634/634 [==============================] - 25s 39ms/step - loss: 3.5902 - accuracy: 0.4151
Epoch 5/100
634/634 [==============================] - 24s 38ms/step - loss: 3.3941 - accuracy: 0.4374
Epoch 6/100
634/634 [==============================] - 23s 36ms/step - loss: 3.2046 - accuracy: 0.4610
Epoch 7/100
634/634 [==============================] - 23s 37ms/step - loss: 3.0249 - accuracy: 0.4828
Epoch 8/100
634/634 [==============================] - 23s 37ms/step - loss: 2.8624 - accuracy: 0.5020
Epoch 9/100
634/634 [==============================] - 21s 34ms/step - loss: 2.7222 - accuracy: 0.5177
Epoch 10/100
634/634 [==============================] - 22s 34ms/step - l

634/634 [==============================] - 21s 34ms/step - loss: 0.3422 - accuracy: 0.9028
Epoch 80/100
634/634 [==============================] - 24s 38ms/step - loss: 0.3401 - accuracy: 0.9043
Epoch 81/100
634/634 [==============================] - 23s 37ms/step - loss: 0.3331 - accuracy: 0.9042
Epoch 82/100
634/634 [==============================] - 22s 34ms/step - loss: 0.3310 - accuracy: 0.9055
Epoch 83/100
634/634 [==============================] - 22s 35ms/step - loss: 0.3233 - accuracy: 0.9080
Epoch 84/100
634/634 [==============================] - 22s 35ms/step - loss: 0.3191 - accuracy: 0.9076
Epoch 85/100
634/634 [==============================] - 24s 38ms/step - loss: 0.3157 - accuracy: 0.9081
Epoch 86/100
634/634 [==============================] - 22s 35ms/step - loss: 0.3105 - accuracy: 0.9100
Epoch 87/100
634/634 [==============================] - 24s 38ms/step - loss: 0.3087 - accuracy: 0.9095
Epoch 88/100
634/634 [==============================] - 25s 39ms/step - loss:

In [15]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 900
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(spanish_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(spa_pad_sentence[index:index+1])[0], eng_text_tokenizer))

The english sentence is: toms safe
The spanish sentence is: tom está seguro
The predicted sentence is :
1/1 [==============================] - 0s 24ms/step
toms tom sure <empty> <empty>


## Reference and Credits

- Nechu BM, **[How to build an encoder decoder translation model using LSTM with Python and Keras](https://towardsdatascience.com/how-to-build-an-encoder-decoder-translation-model-using-lstm-with-python-and-keras-a31e9d864b9b)**